In [5]:
import pandas as pd
import numpy as np

In [6]:
recipes = pd.read_csv('myRec/app/recommender_comp/datasets/new_allrecipes.csv', sep=",", error_bad_lines=False, encoding="latin-1")
recipes

,id,title,category,cook_time_minutes,ingredients,instructions,photo_url,prep_time_minutes,total_time_minutes,rating_stars,review_count,calories,url
0,13277,Cheese Soup II,"Soups, Stews and Chili",0,"['5 cups cubed potatoes', '2 cups carrots, sli...",['In a 4 quart casserole dish combine cubed po...,http://images.media-allrecipes.com/global/reci...,0,0,1.00,0,361,http://allrecipes.com/recipe/13277/
1,13576,Lucy's Carrot Pudding,Desserts,0,"['1/2 cup butter', '1 cup white sugar', '1 cup...",['Cream 1/2 cup butter or margarine and 1 cup ...,http://images.media-allrecipes.com/global/reci...,0,0,1.00,0,470,http://allrecipes.com/recipe/13576/
2,13591,Grilled Goose with Prune Stuffing and Gravy,Meat and Poultry,0,"['1 (10 pound) whole goose', '2 tablespoons ko...",['Rinse goose and pat dry. Remove excess fat. ...,http://images.media-allrecipes.com/global/reci...,0,0,1.00,0,813,http://allrecipes.com/recipe/13591/
3,13786,Sweet and Sour Sauce For Gingerbread,Side Dish,0,"['1 cup packed brown sugar', '1 cup white suga...","['In a saucepan, combine the brown sugar, whit...",http://images.media-allrecipes.com/global/reci...,0,0,1.00,0,103,http://allrecipes.com/recipe/13786/
4,13857,White Potato Dressing,Side Dish,0,"['10 pounds white potatoes, peeled and quarter...",['Preheat oven to 350 degrees F (175 degrees C...,http://images.media-allrecipes.com/global/reci...,0,0,1.00,0,495,http://allrecipes.com/recipe/13857/
5,15270,Fruit Meringues,Desserts,0,"['4 egg whites', '1/4 teaspoon cream of tartar...",['Preheat oven to 275 degrees F (135 degrees C...,http://images.media-allrecipes.com/global/reci...,0,0,1.00,0,92,http://allrecipes.com/recipe/15270/
6,16213,Sherry Chiffon Pie,Desserts,0,['1 (9 inch) prepared chocolate cookie crumb c...,['Soften gelatin in cold water for 5 minutes.'...,http://images.media-allrecipes.com/global/reci...,0,0,1.00,0,280,http://allrecipes.com/recipe/16213/
7,17189,Nesselrode Pie II,Desserts,30,"['1 (9 inch) prepared graham cracker crust', '...","['In a 1 quart saucepan, stir gelatin with sal...",http://images.media-allrecipes.com/global/reci...,0,130,1.00,0,341,http://allrecipes.com/recipe/17189/
8,17364,Christmas Pie,Desserts,20,"['1 (9 inch) pie shell, baked', '1/4 cup water...",['Pour cold water into a small bowl. Sprinkle ...,http://images.media-allrecipes.com/global/reci...,0,140,1.00,0,353,http://allrecipes.com/recipe/17364/
9,17384,Mock Pecan Pie II,Desserts,10,"['1 (9 inch) unbaked pie crust', '1/4 cup unsa...",['Preheat oven to 350 degrees F (175 degrees C...,http://images.media-allrecipes.com/global/reci...,45,55,1.00,0,384,http://allrecipes.com/recipe/17384/


In [7]:
recipes['ingredients'].head()

0    ['5 cups cubed potatoes', '2 cups carrots, sli...
1    ['1/2 cup butter', '1 cup white sugar', '1 cup...
2    ['1 (10 pound) whole goose', '2 tablespoons ko...
3    ['1 cup packed brown sugar', '1 cup white suga...
4    ['10 pounds white potatoes, peeled and quarter...
Name: ingredients, dtype: object

In [8]:
recipes['instructions'].head()

0    ['In a 4 quart casserole dish combine cubed po...
1    ['Cream 1/2 cup butter or margarine and 1 cup ...
2    ['Rinse goose and pat dry. Remove excess fat. ...
3    ['In a saucepan, combine the brown sugar, whit...
4    ['Preheat oven to 350 degrees F (175 degrees C...
Name: instructions, dtype: object

###  Compute Term Frequency-Inverse Document Frequency (TF-IDF) vectors for each document
This will give you a matrix where each column represents a word in the overview vocabulary (all the words that appear in at least one document) and each column represents a recipe, as before.

TF-IDF score is the frequency of a word occurring in a document, down-weighted by the number of documents in which it occurs. This is done to reduce the importance of words that occur frequently in plot overviews and therefore, their significance in computing the final similarity score

In [9]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

In [11]:
#Replace NaN with an empty string
recipes['ingredients'] =recipes['ingredients'].fillna('')
recipes['ingredients'].head()

0    ['5 cups cubed potatoes', '2 cups carrots, sli...
1    ['1/2 cup butter', '1 cup white sugar', '1 cup...
2    ['1 (10 pound) whole goose', '2 tablespoons ko...
3    ['1 cup packed brown sugar', '1 cup white suga...
4    ['10 pounds white potatoes, peeled and quarter...
Name: ingredients, dtype: object

In [12]:
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(recipes['ingredients'])

## 1621 words were used to describe ingredients, for 5422 recipes

In [13]:
#Output the shape of tfidf_matrix
tfidf_matrix.shape

(5422, 1621)

You will be using the cosine similarity to calculate a numeric quantity that denotes the similarity between two movies. You use the cosine similarity score since it is independent of magnitude and is relatively easy and fast to calculate (especially when used in conjunction with TF-IDF scores, which will be explained later). Mathematically, it is defined as follows: 

cosine(x,y)=x.y⊺||x||.||y||cosine(x,y)=x.y⊺||x||.||y||
Since you have used the TF-IDF vectorizer, calculating the dot product will directly give you the cosine similarity score. Therefore, you will use sklearn's linear_kernel() instead of cosine_similarities() since it is faster.

In [14]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

In [15]:
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

You're going to define a function that takes in a movie title as an input and outputs a list of the 10 most similar movies. Firstly, for this, you need a reverse mapping of movie titles and DataFrame indices. In other words, you need a mechanism to identify the index of a movie in your metadata DataFrame, given its title.

In [16]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(recipes.index, index=recipes['title']).drop_duplicates()

- Get the index of the movie given its title.
- Get the list of cosine similarity scores for that particular movie with all movies. Convert it into a list of tuples where the first element is its position and the second is the similarity score.
- Sort the aforementioned list of tuples based on the similarity scores; that is, the second element.
- Get the top 10 elements of this list. Ignore the first element as it refers to self (the movie most similar to a particular movie is the movie itself).
- Return the titles corresponding to the indices of the top elements.

In [17]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    recipe_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return recipes['title'].iloc[recipe_indices]

In [18]:
get_recommendations('Banana Crumb Muffins')

1009               Banana Split Cookies
2586             Cinnamon Sugar Cookies
4344         Janine's Best Banana Bread
2065                    Pineapple Puffs
717                  Butterscotch Bread
5007    Grandma's Homemade Banana Bread
2411             Lighter Banana Muffins
4214                    Streusel Kuchen
2959                Finnish Pannu Kakku
5054                Banana Bran Muffins
Name: title, dtype: object

In [19]:
get_recommendations('Stuffed Mushrooms IV')

2094    Best Ever Meatloaf with Brown Gravy
3856                      Mushroom Meatloaf
3970                       Mushroom Risotto
1924                            Peanut Soup
2985                   Tomato-Mushroom Soup
4712            Oven Fried Parmesan Chicken
4534       Mouth-Watering Stuffed Mushrooms
3105       Mushroom Stuffed Chicken Rollups
5160                         Mushroom Sauce
3021                       Creamy Corn Soup
Name: title, dtype: object

In [20]:
get_recommendations('Maple Roast Turkey')

4517        Maple Roast Turkey and Gravy
2340     Awesome Tangerine-Glazed Turkey
4813                   Vegetable Chowder
4533            Beef and Barley Soup III
3667                   Oyster Dressing I
4862    Ibby's Pumpkin Mushroom Stuffing
1874                 Veggie Cheddar Soup
4024                 Chicken Jambalaya I
1433                   Mushroom Stuffing
5387               Mulligatawny Soup III
Name: title, dtype: object

# Credits, Genres and Keywords Based Recommender

# the 3 top actors, the director, related genres and the movie plot keywords.

# => category, instructions keywords 

In [29]:
description = pd.read_csv('myRec/app/recommender_comp/datasets/data_description.csv', sep=",", error_bad_lines=False, encoding="latin-1")
description

,id,item_id,author,cook_time_minutes,description,footnotes,ingredients,instructions,photo_url,prep_time_minutes,rating_stars,review_count,time_scraped,title,total_time_minutes,url
0,6664,6664/,Stephanie,25,I just started adding my favorite things to ba...,[],"['1/2 cup unsalted butter, chilled and cubed',...",['Preheat oven to 400 degrees F (205 degrees C...,http://images.media-allrecipes.com/userphotos/...,55,4.32,46,1498204021,"Basil, Roasted Peppers and Monterey Jack Cornb...",100,http://allrecipes.com/Recipe/6664/
1,6663,6663/,Stephanie,0,These are great as an appetizer or served alon...,[],"['1/2 cup Parmesan cheese', '3/4 teaspoon grou...","['Combine parmesan cheese, pepper and garlic p...",http://images.media-allrecipes.com/userphotos/...,0,4.18,44,1498204301,Crispy Cheese Twists,0,http://allrecipes.com/Recipe/6663/
2,6665,6665/,Kathy,0,This is the best bread recipe. Light and fluf...,[],"['2 cups hot water', '1/2 cup margarine', '1/3...",['Melt margarine in hot water. Add sugar and s...,http://images.media-allrecipes.com/userphotos/...,0,3.65,168,1498206088,Mom's Yeast Rolls,0,http://allrecipes.com/Recipe/6665/
3,6666,6666/,Mary E. Crain,0,A Southern delight with a 'more-ish' taste. ...,[],"['1 1/2 cups white sugar', '1/2 cup vegetable ...",['Combine sugar and oil; beat well. Add eggs a...,http://images.media-allrecipes.com/userphotos/...,0,4.70,344,1498209193,Sweet Potato Bread I,0,http://allrecipes.com/Recipe/6666/
4,6668,6668/,DeeDee,45,You can make these into braids or buns. Frost ...,"[""Cook's Note:"", 'I made 1 braid, 12 cinnamon ...","['1/4 cup butter', '1 teaspoon white sugar', '...",['Stir butter and 1 teaspoon sugar into the ho...,http://images.media-allrecipes.com/userphotos/...,20,4.71,6,1498205251,Orange Buns,170,http://allrecipes.com/Recipe/6668/
5,6667,6667/,Linda Jackman,0,Delicious! This recipe originated in Cornwall...,[],"['1 teaspoon active dry yeast', '1 1/4 cups lu...","['In a small bowl, dissolve the yeast in the m...",http://images.media-allrecipes.com/userphotos/...,0,4.33,88,1498205958,Cornish Splits,0,http://allrecipes.com/Recipe/6667/
6,6669,6669/,DeeDee,0,A rich and spicy bread!,[],"['8 cups all-purpose flour', '4 cups shredded ...","['In a very large bowl, combine 7 cups of flou...",http://images.media-allrecipes.com/userphotos/...,0,4.33,86,1498205490,Jalapeno Cheese Bread,0,http://allrecipes.com/Recipe/6669/
7,6670,6670/,Dee Dee,0,"Whole wheat bread with sunflower seeds, cracke...",['Aluminum foil can be used to keep food moist...,"['2 tablespoons active dry yeast', '1 teaspoon...","['In a small bowl, dissolve the yeast and suga...",http://images.media-allrecipes.com/userphotos/...,0,4.81,133,1498204889,Dee's Health Bread,0,http://allrecipes.com/Recipe/6670/
8,6671,6671/,DeeDee,45,Dough needs to rise overnight.,[],"['1 cup rolled oats', '1/2 cup molasses', '1/3...","['Combine oats, molasses, oil, salt and boilin...",http://images.media-allrecipes.com/userphotos/...,30,4.59,29,1498206052,Oatmeal Bread I,0,http://allrecipes.com/Recipe/6671/
9,6672,6672/,Mary E. Crain,0,A bread you MUST try! This is one you'll want...,[],"['3 cups all-purpose flour', '2 cups white sug...",['Grease and flour a 9 x 5 inch pan well. Preh...,http://images.media-allrecipes.com/userphotos/...,0,3.95,36,1498205368,Strawberry Bread I,0,http://allrecipes.com/Recipe/6672/


In [30]:
description['id'] = description['id'].astype('int')
recipes = pd.merge(recipes, description.iloc[:, [0,4]], how='left', on='id')
recipes.head()

,id,title,category,cook_time_minutes,ingredients,instructions,photo_url,prep_time_minutes,total_time_minutes,rating_stars,review_count,calories,url,description_x,description_y,description_x,description_y
0,13277,Cheese Soup II,"Soups, Stews and Chili",0,"['5 cups cubed potatoes', '2 cups carrots, sli...",['In a 4 quart casserole dish combine cubed po...,http://images.media-allrecipes.com/global/reci...,0,0,1.0,0,361,http://allrecipes.com/recipe/13277/,This is my mom's recipe that she has adapted f...,This is my mom's recipe that she has adapted f...,This is my mom's recipe that she has adapted f...,This is my mom's recipe that she has adapted f...
1,13576,Lucy's Carrot Pudding,Desserts,0,"['1/2 cup butter', '1 cup white sugar', '1 cup...",['Cream 1/2 cup butter or margarine and 1 cup ...,http://images.media-allrecipes.com/global/reci...,0,0,1.0,0,470,http://allrecipes.com/recipe/13576/,Our family likes this dessert during the holid...,Our family likes this dessert during the holid...,Our family likes this dessert during the holid...,Our family likes this dessert during the holid...
2,13591,Grilled Goose with Prune Stuffing and Gravy,Meat and Poultry,0,"['1 (10 pound) whole goose', '2 tablespoons ko...",['Rinse goose and pat dry. Remove excess fat. ...,http://images.media-allrecipes.com/global/reci...,0,0,1.0,0,813,http://allrecipes.com/recipe/13591/,Grilling the bird makes the most wonderfully m...,Grilling the bird makes the most wonderfully m...,Grilling the bird makes the most wonderfully m...,Grilling the bird makes the most wonderfully m...
3,13786,Sweet and Sour Sauce For Gingerbread,Side Dish,0,"['1 cup packed brown sugar', '1 cup white suga...","['In a saucepan, combine the brown sugar, whit...",http://images.media-allrecipes.com/global/reci...,0,0,1.0,0,103,http://allrecipes.com/recipe/13786/,This cooked icing is wonderful served over sli...,This cooked icing is wonderful served over sli...,This cooked icing is wonderful served over sli...,This cooked icing is wonderful served over sli...
4,13857,White Potato Dressing,Side Dish,0,"['10 pounds white potatoes, peeled and quarter...",['Preheat oven to 350 degrees F (175 degrees C...,http://images.media-allrecipes.com/global/reci...,0,0,1.0,0,495,http://allrecipes.com/recipe/13857/,This dressing recipe substitutes potatoes for ...,This dressing recipe substitutes potatoes for ...,This dressing recipe substitutes potatoes for ...,This dressing recipe substitutes potatoes for ...
